In [1]:
from __future__ import division
import warnings; warnings.simplefilter('ignore') #for PCoA warnings
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from DEICODE import untangle, fetch, optimize_impute
from biom import load_table
#transforms 
from gneiss.util import match
from skbio.stats.composition import clr, ilr, centralize
#else
%matplotlib inline

In [2]:
map_file='data/benchmarking/map.txt' #import metadata
in_biom='data/benchmarking/cfs.biom' #import biom file
table = load_table(in_biom)
mapping = pd.read_table(map_file, index_col=0)

table = pd.DataFrame(np.array(table.matrix_data.todense()).T, 
                     index=table.ids(axis='sample'),
                     columns=table.ids(axis='observation'))
table, mapping = match(table, mapping)

In [3]:
from gneiss.cluster import correlation_linkage

formula = "Subject+Sex+Age+BMI+sCD14ugml+LPSpgml"
tree = correlation_linkage(table+1)
res_pseudo, params, cv_pseudo = optimize_impute.ols_regression(
    table+1, tree, mapping, 
    formula=formula, complete_matrix=False)

In [4]:
res_complete, params, cv_complete = optimize_impute.ols_regression(
    table, tree, mapping, 
    formula=formula, complete_matrix=True)

In [5]:
import pickle

pickle.dump(res_pseudo, open('cfs_pseudo_regression_model.pickle', 'wb'))
cv_pseudo.to_csv('cfs_pseudo_cv.csv')
pickle.dump(res_complete, open('cfs_complete_regression_model.pickle', 'wb'))
cv_complete.to_csv('cfs_complete_cv.csv')